# Collect DFT data into *, *O, *OH collections
---

# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import pickle

import pandas as pd
# pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)
# pd.options.display.max_colwidth = 100
import numpy as np

# #########################################################
from IPython.display import display

# #########################################################
from methods import get_df_jobs_anal
from methods import get_df_jobs_data

# #########################################################
from local_methods import calc_ads_e

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/dft_workflow/job_analysis/collect_collate_dft_data
Divisor is Energy class instance!!!
Divisor is Energy class instance!!!


# Script Inputs

In [2]:
verbose = False
# verbose = True

# Read Data

In [3]:
df_jobs_anal = get_df_jobs_anal()

df_jobs_data = get_df_jobs_data()

# Filtering dataframe for testing

In [4]:
# print("TEMP")

# df_index = df_jobs_anal.index.to_frame()

# df_jobs_anal = df_jobs_anal.loc[
#     df_index[df_index.compenv == "slac"].index
#     ]

In [5]:
# #########################################################
# Only completed jobs will be considered
df_jobs_anal_i = df_jobs_anal[df_jobs_anal.job_completely_done == True]

# #########################################################
# Remove the *O slabs for now
# The fact that they have NaN active sites will mess up the groupby
ads_list = df_jobs_anal_i.index.get_level_values("ads").tolist()
ads_list_no_o = [i for i in list(set(ads_list)) if i != "o"]

idx = pd.IndexSlice
df_jobs_anal_no_o = df_jobs_anal_i.loc[idx[:, :, ads_list_no_o, :, :], :]

# Main Loop

In [6]:
# #########################################################
verbose_local = False
# #########################################################

data_dict_list = []
groupby_cols = ["compenv", "slab_id", "active_site", ]
grouped = df_jobs_anal_no_o.groupby(groupby_cols)
for name_i, group in grouped:
# for i in range(1):

    # group = grouped.get_group(
    #     ('slac', 'fagumoha_68', 63.0)
    #     )

    if verbose_local:
        print(40 * "*")
        print(name_i)

    data_dict_i = dict()

    # #####################################################
    name_dict_i = dict(zip(groupby_cols, name_i))
    # #####################################################
    compenv_i = name_i[0]
    slab_id_i = name_i[1]
    active_site_i = name_i[2]
    # #####################################################

    # Selecting the relevent *O slab rows and combining with group
    idx = pd.IndexSlice
    df_o_slabs = df_jobs_anal_i.loc[idx[compenv_i, slab_id_i, "o", :, :], :]

    group_i = pd.concat([
        df_o_slabs,
        group
        ])

    data_dict_list_j = []
    for name_j, row_j in group_i.iterrows():
        data_dict_j = dict()

        # #################################################
        name_dict_j = dict(zip(list(group_i.index.names), name_j))
        # #################################################
        job_id_max_j = row_j.job_id_max
        # #################################################

        # #################################################
        row_data_j = df_jobs_data.loc[job_id_max_j]
        # #################################################
        pot_e_j = row_data_j.pot_e
        # #################################################

        # #################################################
        data_dict_j.update(name_dict_j)
        data_dict_j["pot_e"] = pot_e_j
        data_dict_j["job_id_max"] = job_id_max_j
        # #################################################
        data_dict_list_j.append(data_dict_j)
        # #################################################

    df_tmp = pd.DataFrame(data_dict_list_j)
    df_tmp = df_tmp.set_index(
        ["compenv", "slab_id", "ads", "active_site", ],
        drop=False)

    # #####################################################
    df_ads_o = df_tmp[df_tmp.ads == "o"]
    df_ads_oh = df_tmp[df_tmp.ads == "oh"]
    df_ads_bare = df_tmp[df_tmp.ads == "bare"]

    if (df_ads_o.shape[0] > 1) or (df_ads_oh.shape[0] > 1) or (df_ads_bare.shape[0] > 1):
        if verbose_local:
            print("There is more than 1 row per state here, need a better way to select")
    # #####################################################


    # If there isn't a bare * calculation then skip for now
    if df_ads_bare.shape[0] == 0:
        if verbose_local:
            print("No bare slab available")
        continue

    df_ads_oh = df_ads_oh[df_ads_oh.pot_e == df_ads_oh.pot_e.min()]
    df_ads_o = df_ads_o[df_ads_o.pot_e == df_ads_o.pot_e.min()]
    df_ads_bare = df_ads_bare[df_ads_bare.pot_e == df_ads_bare.pot_e.min()]

    job_id_bare_i = df_ads_bare.iloc[0].job_id_max

    df_ads_i = pd.concat([
        df_ads_o,
        df_ads_oh,
        df_ads_bare])
    df_ads_i = calc_ads_e(df_ads_i)



    # #########################################################
    row_oh_i = df_ads_i[df_ads_i.ads == "oh"]
    if row_oh_i.shape[0] == 1:
        row_oh_i = row_oh_i.iloc[0]
        # #####################################################
        ads_e_oh_i = row_oh_i.ads_e
        job_id_oh_i = row_oh_i.job_id_max
        # #####################################################
    else:
        ads_e_oh_i = None
        job_id_oh_i = None

    # #########################################################
    row_o_i = df_ads_i[df_ads_i.ads == "o"]
    if row_o_i.shape[0] == 1:
        row_o_i = row_o_i.iloc[0]
        # #####################################################
        ads_e_o_i = row_o_i.ads_e
        job_id_o_i = row_o_i.job_id_max
        # #####################################################
    else:
        ads_e_o_i = None
        job_id_o_i = None



    if verbose_local:
        print(80 * "#")




    # #####################################################
    data_dict_i.update(name_dict_i)
    data_dict_i["g_o"] = ads_e_o_i
    data_dict_i["g_oh"] = ads_e_oh_i
    data_dict_i["job_id_o"] = job_id_o_i
    data_dict_i["job_id_oh"] = job_id_oh_i 
    data_dict_i["job_id_bare"] = job_id_bare_i
    # #####################################################
    data_dict_list.append(data_dict_i)
    # #####################################################


    # display(df_tmp_1)
    # if df_ads_i.shape[0] == 3:
    #     break


    if verbose_local:
        print("")

# #########################################################
df_ads = pd.DataFrame(data_dict_list)

# df_ads.iloc[0:3]

****************************************
('nersc', 'fosurufu_23', 43.0)
################################################################################

****************************************
('nersc', 'fosurufu_23', 45.0)
################################################################################

****************************************
('nersc', 'galopuba_86', 21.0)
################################################################################

****************************************
('nersc', 'hadogato_47', 84.0)
################################################################################

****************************************
('nersc', 'hadogato_47', 86.0)
################################################################################

****************************************
('nersc', 'hadogato_47', 87.0)
################################################################################

****************************************
('nersc', 'hadogato_47', 88.0)
####

In [13]:
df_ads_i = df_ads[~df_ads.g_oh.isnull()]

# print(df_ads_i.shape)

df_ads_i

,compenv,slab_id,active_site,g_o,g_oh,job_id_o,job_id_oh,job_id_bare
20,sherlock,fogalonu_46,16.0,2.559308,1.124083,kagehano_47,rataruha_18,bebogeni_65
21,sherlock,kenukami_73,84.0,4.444919,-0.008826,tinusohi_68,gawopigo_86,folelule_90
22,sherlock,kenukami_73,86.0,5.221788,0.328082,tinusohi_68,pagugovi_98,rakisefe_18
23,sherlock,kipatalo_90,78.0,2.861919,0.922864,tolivogi_84,masoroko_92,mawenumo_32
24,sherlock,kipatalo_90,81.0,2.843177,1.158436,tolivogi_84,rahulesi_46,waduhulu_59
25,sherlock,rakawavo_17,25.0,2.074328,0.380545,rasogohe_76,kidovase_29,dedutabo_58
26,sherlock,tofiwadi_49,45.0,3.513025,0.694920,robowulu_21,furugahi_30,pibawima_72
27,sherlock,tofiwadi_49,46.0,3.655949,0.964720,robowulu_21,gawusomo_30,guromotu_58
28,sherlock,tofiwadi_49,47.0,3.638215,0.910759,robowulu_21,hereweso_08,fohuviha_77
29,sherlock,vuvunira_55,65.0,3.259376,0.273765,bubadowi_91,dusanala_91,fabofapa_68


In [8]:
# assert False

In [9]:
# Pickling data ###########################################
directory = os.path.join(
    os.environ["PROJ_irox_oer"],
    "dft_workflow/job_analysis/collect_collate_dft_data",
    "out_data")
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "df_ads.pickle"), "wb") as fle:
    pickle.dump(df_ads, fle)
# #########################################################

In [10]:
from methods import get_df_ads

df_ads_tmp = get_df_ads()

df_ads_tmp.head()

,compenv,slab_id,active_site,g_o,g_oh,job_id_o,job_id_oh,job_id_bare
0,nersc,fosurufu_23,43.0,3.752880,NaN,sufarabe_10,None,mamopaba_02
1,nersc,fosurufu_23,45.0,4.064136,NaN,sufarabe_10,None,tikuboli_46
2,nersc,galopuba_86,21.0,3.273659,NaN,wipugigu_63,None,gedihoki_54
3,nersc,hadogato_47,84.0,4.186781,NaN,fawehihu_36,None,mohonopa_88
4,nersc,hadogato_47,86.0,3.503503,NaN,fawehihu_36,None,gugobine_21


In [11]:
# #########################################################
print(20 * "# # ")
print("All done!")
print("analyse_jobs.ipynb")
print(20 * "# # ")
# #########################################################

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
All done!
analyse_jobs.ipynb
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 


In [12]:
# df_ads

# assert False